<a href="https://colab.research.google.com/github/RawanKhaled20/Huawei.Tasks.DeepLearning/blob/main/Task1_Huawei.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import the required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Conv1D, GlobalMaxPooling1D, Dense, Dropout , LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import tensorflow as tf
import re
from tensorflow.keras.regularizers import l2
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.optimizers import legacy

In [2]:
print(tf.test.gpu_device_name())

In [ ]:
from google.colab import files
files.upload()

In [5]:
imdb = pd.read_csv("/content/IMDB Dataset.csv")

In [6]:
imdb.drop_duplicates(inplace=True)

In [7]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
texts = imdb['review'].values
labels = imdb['sentiment'].values

In [12]:
# Preprocess the reviews
stop_words = set(stopwords.words('english'))
processed_texts = []

for text in texts:
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove special characters, URLs, and email addresses
    text = re.sub(r"[^a-zA-Z0-9]+", ' ', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)

    # Remove numbers
    text = re.sub(r"\d+", "", text)

    # Convert to lower case and tokenize
    text = word_tokenize(text.lower())

    # Remove stop words
    text = [word for word in text if not word in stop_words]

    # Remove repeated characters and single characters
    processed_text = []

    for word in text:
        word = re.sub(r'\b(\w*?)(\w)\2+(\w*?\b|\b)', r'\1\2\3', word)
        if len(word) > 1:
            processed_text.append(word)
    processed_texts.append(' '.join(processed_text))

processed_texts[5]

'probably time favorite movie story selflesness sacrifice dedication noble cause preachy boring never gets old despite sen times last years paul lukas performance brings tears eyes bete davis one truly sympathetic roles delight kids grandma says like dresed midgets children makes fun watch mother slow awakening hapening world rof believable startling dozen thumbs movie'

In [13]:
max_words = 10000
max_length = 500
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(processed_texts)
sequences = tokenizer.texts_to_sequences(processed_texts)
X = pad_sequences(sequences, maxlen=max_length)
y = np.array(imdb['sentiment'].map({'positive': 1, 'negative': 0}))

In [14]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
with tf.device('/GPU:0'):

    # Build the neural network
    model = Sequential([
        Embedding(input_dim=10000, output_dim=128, input_length=500),
        #GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        #Bidirectional(GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
        #Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
        Conv1D(128, 5, activation='relu'),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')])

# Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    # Train the model
    early_stopping = EarlyStopping(patience=3)
    model.fit(X_train, y_train, epochs=2, batch_size=128, validation_split=0.1, callbacks=[early_stopping])
    _, accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {accuracy}")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 conv1d_2 (Conv1D)           (None, 496, 128)          82048     
                                                                 
 conv1d_3 (Conv1D)           (None, 492, 128)          82048     
                                                                 
 global_max_pooling1d_1 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                      

In [34]:
# Assuming you want to get the first sequence
sequence_index = 8  # Change this index as needed

single_sequence = X[sequence_index]

single_sequence=single_sequence.reshape(1, -1)
predictions = model.predict(single_sequence)
# You can now work with the 'predictions' array, which contains the model's predictions.
print(predictions)

threshold = 0.5
if predictions >= threshold:
    prediction_text = "positive"
else:
    prediction_text = "negative"

print(f"The prediction is {prediction_text}.")

1/1 [==============================] - 0s 27ms/step
[[0.48193532]]
The prediction is negative.
